In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

### ThoT Building Prompt Generator  

Based on "The Prompt Report: A Systematic Survey of Prompting Techniques.",
a detailed vocabulary of 33 vocabulary terms, a taxonomy of 58 LLM prompting techniques, and 40 techniques for other modalities.

**Thread-of-Thought (ThoT)** Prompting (Zhouet al., 2023)

consists of an improved thought inducer for CoT reasoning. Instead of "Let’s think step by step," it uses "Walk me through this context in manageable parts step by step, summarizing and analyzing as we go." This thought inducer works
well in question-answering and retrieval settings,especially when dealing with large, complex contexts.ThoT is a part of CoT (chain of thoughts)

**Chain-of-Thought (CoT)** is a prompt engineering framework that enables reasoning
capabilities through intermediate steps. There are various sub-techniques of CoT including
self-consistency, active-prompt, and multimodal CoT that each have strengths and
weaknesses depending on the specific application



#### **Goal**

The main idea is to develop automated **prompt generator agents**. These agents will be designed to:

* Analyze a given question.
* Generate an optimized prompt using various prompting techniques.

The development of these agents relies on:

* Recent prompt techniques, defined and classified in the document **"The Prompt Report: A Systematic Survey of Prompting Techniques"**.

#### **Functionality**

1.  **Question Reception:** The agent receives a question as input.
2.  **Question Analysis:** The agent analyzes the question to understand its intent and requirements.
3.  **Prompt Approach Selection:** Based on the analysis and the techniques from "The Prompt Report," the agent selects the most appropriate prompting approach.
4.  **Prompt Generation:** The agent generates an optimized prompt using the selected approach.

#### **Expected Benefits**

* Facilitate the creation of complex prompts.
* Optimize the efficiency of AI models.
* Make the use of AI more accessible.






In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-rPBnsXimJy GIZTWPKAtav6nWikZPSewB3oeo055nFcLW9rnMorepgfeAZNQQz5FUUwrtmQsA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
# Définition du prompt
thot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Thread-of-Thought (ThoT) prompts based on user questions.
    
    (ThoT) is a part of Chain-of-Thought (CoT) prompting leverages few-shot prompting to encourage the LLM to express its thought process before delivering its final answer.
    
    This technique enhances the LLM's performance in reasoning tasks.
    
    A CoT prompt includes an exemplar featuring a question, a reasoning path, and the correct answer.
    Thread-of-Thought (ThoT) is a specialized form of CoT.
    The ThoT process includes the following steps:
    1. High-level prompt: "Walk me through this context in manageable parts step by step, summarizing and analyzing as we go."
    2. Summarize and analyze: Break down the question's context into manageable parts, summarizing and analyzing each part.
    3. Answer the original question: Provide a final answer based on the analysis.
    
    Please structure your response in the following format:
    
    Step 1: [High-level prompt]
    Step 2: [Summarize and analyze each part]
    Step 3: [Final Answer]
    
    Example:
    Question: "What are the main causes of climate change?"
    
    Step 1: Walk me through this context in manageable parts step by step, summarizing and analyzing as we go.
    
    Step 2: 
    Thread 1 - Understanding the concept of climate change:
    Climate change refers to significant, long-term changes in the global climate. The Earth's average surface temperature has increased by about 1.1°C since the pre-industrial era. This increase might seem small, but even minor changes in average global temperature can drastically affect weather patterns, ecosystems, and human societies.
    
    Thread 2 - Examining fossil fuel combustion:
    The primary driver of climate change is the burning of fossil fuels (coal, oil, and natural gas). When we analyze this process, fossil fuels release carbon dioxide (CO2) that has been stored underground for millions of years. This CO2 enters the atmosphere and enhances the greenhouse effect. Transportation, electricity generation, and industrial processes are the major sectors contributing to these emissions. The data shows that CO2 concentrations have increased from about 280 parts per million (ppm) in pre-industrial times to over 415 ppm today.
    
    Thread 3 - Analyzing deforestation impacts:
    Forests act as carbon sinks, absorbing CO2 from the atmosphere. When forests are cleared (often for agriculture, timber, or urban development), this stored carbon is released back into the atmosphere. Additionally, fewer trees mean reduced capacity to remove CO2 from the air. Tropical deforestation alone accounts for about 10% of human-caused greenhouse gas emissions. Satellite imagery reveals we lose approximately 10 million hectares of forest annually.
    
    Thread 4 - Exploring agricultural and industrial activities:
    Beyond CO2, other greenhouse gases play significant roles. Methane (CH4), released from livestock, rice paddies, and landfills, has a warming potential 25 times greater than CO2 over a 100-year period. Nitrous oxide (N2O) from fertilizers and industrial processes is about 300 times more potent than CO2. Industrial processes also emit fluorinated gases with extremely high warming potentials, some thousands of times more potent than CO2.
    
    Thread 5 - Connecting the mechanisms:
    These greenhouse gases function by allowing sunlight to enter the atmosphere but trapping the reflected heat, similar to how a greenhouse operates. This trapped heat leads to rising global temperatures, which in turn causes melting ice caps, rising sea levels, altered precipitation patterns, and increased frequency of extreme weather events. Climate models demonstrate that the observed warming cannot be explained by natural factors alone and correlates strongly with human activities.
    
    Step 3: The main causes of climate change are human activities, primarily the burning of fossil fuels (releasing CO2), deforestation (reducing carbon absorption and releasing stored carbon), and agricultural/industrial processes (emitting methane, nitrous oxide, and other potent greenhouse gases). These activities intensify the greenhouse effect by increasing atmospheric concentrations of heat-trapping gases, leading to global warming and its associated impacts on our planet's climate systems.
    """),
    ("human", "{question}"),
    ("assistant", """
    Walk me through this context in manageable parts step by step, summarizing and analyzing as we go:
    
    Thread 1 - [First logical component of analysis]:
    [Detailed examination of the first aspect of the question, breaking down key concepts, analyzing relationships, and providing relevant context and examples]
    
    Thread 2 - [Second logical component of analysis]:
    [Detailed examination of the second aspect, building upon previous insights while introducing new perspectives and data points]
    
    Thread 3 - [Third logical component of analysis]:
    [Detailed examination of the third aspect, further deepening the analysis with additional considerations and potentially addressing counterpoints]
    
    [Additional threads as needed depending on question complexity]
    
    Thread n - [Final integrative component]:
    [Synthesis of all threads, connecting insights across the different components of analysis and drawing overarching conclusions]
    
    Final Answer: [Concise, comprehensive answer that integrates the complete analysis while directly addressing the original question]
    """)
])

In [6]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the ThoT prompt and extract the final answer (Step 3)
def generate_thot_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = thot_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from Step 3
    final_answer_pattern = r'Step 3:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 3: \[Final Answer\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('thot_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (STEP 3) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_thot", generate_thot_node)

# Graph configuration
graph_builder.set_entry_point("generate_thot")
graph_builder.add_edge("generate_thot", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="Explain Jean-Jacques Rousseau.")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER (STEP 3):")

# Graph execution - only display Step 3
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_thot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'thot_result.txt')")

QUESTION:
Explain Jean-Jacques Rousseau.

FINAL ANSWER (STEP 3):
Jean-Jacques Rousseau was a pivotal Enlightenment thinker whose ideas on the social contract, the nature of humanity, education, and inequality have had a lasting impact on political philosophy and educational theory. He believed that humans are inherently good but are corrupted by society, advocating for a political system based on the general will and emphasizing the importance of natural education. His critiques of social inequality continue to resonate in modern discussions about justice and governance.

(Complete results saved in 'thot_result.txt')
